# CPE Data
CPE Data

In [1]:
%%capture
!mkdir -p jsondata
%cd jsondata
!rm *.json 
!rm *.zip 
!wget https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-{2002..2022}.json.zip 
!unzip -o "*.zip" 

In [2]:
import glob
import logging
import json
import os
import pandas as pd
import warnings

logging.getLogger('matplotlib.font_manager').disabled = True
warnings.filterwarnings("ignore")

In [3]:
row_accumulator = []


for filename in glob.glob('nvdcve-1.1-20*.json'):
    with open(filename, 'r', encoding='utf-8') as f:
        nvd_data = json.load(f)
        for cves in nvd_data['CVE_Items']:
            if cves['configurations']['nodes'] != []:
                try:
                    for j in range(len(cves['configurations']['nodes'][0])):
                        for cpe in cves['configurations']['nodes'][j]['cpe_match']:
                                if cpe['vulnerable'] == True:
                                    cve = cves['cve']['CVE_data_meta']['ID']
                                    cpe_string = cpe['cpe23Uri']
                                    new_row = { 
                                    'CVE': cve,
                                    'CPE' : cpe_string,
                                    }
                                    row_accumulator.append(new_row)
                except IndexError:
                    pass
                if "children" in cves['configurations']['nodes'][0]:
                    for j in range(len(cves['configurations']['nodes'][0]['children'])):
                        for cpe in cves['configurations']['nodes'][0]['children'][j]['cpe_match']:
                            if cpe['vulnerable'] == True:
                                        cve = cves['cve']['CVE_data_meta']['ID']
                                        cpe_string = cpe['cpe23Uri'] 
                                        new_row = { 
                                        'CVE': cve,
                                        'CPE' : cpe_string,
                                        }
                                        row_accumulator.append(new_row)
        

  
nvd = pd.DataFrame(row_accumulator)                            

In [4]:
nvd[['Format', 'CPE_Version', 'Part', 'Vendor', 'Product', 'Version', 'Update', 'Edition', 'SW_Edition', 'Target_SW', 'Target_HW', 'Language', 'Other']] = nvd['CPE'].str.split(r'(?<!\\):', regex=True, expand=True)



In [ ]:
from pandas_profiling import ProfileReport
profile = ProfileReport(nvd)
profile.to_file("Analysis.pdf")
profile

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]